# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

---

I am going back to the **brochure** idea :
- no chat but input/output
- an output translated in french thanks to Claude (agent)
- with a picture thanks to dall.e (agent)

In [16]:
import os
import json
import requests
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import anthropic
import gradio as gr

## connections :

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-


In [2]:
MODEL = 'gpt-4o-mini'
openai = OpenAI()

claude = anthropic.Anthropic()

## understanding webpages

In [3]:
# A class to represent a webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
#ed.get_contents()
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/

Now, we filter relevant links :

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [6]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt
# print(get_links_user_prompt(ed))

In [13]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

get_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'outsmart page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'posts page', 'url': 'https://edwarddonner.com/posts/'}]}

## Brochure

In [11]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [7]:
def get_brochure_system_prompt(mode):
    """
    create the brochure system prompt
    mode : "serious" or funny
    returns the desired system prompt for creating the brochure
    """

    system_prompt = f"You are an assistant that analyzes the contents of several relevant pages from a company website \n"

    if mode == "serious":
        system_prompt+= f"and creates a short brochure about the company for prospective customers, investors and recruits. \n"

    if mode == "funny":
        system_prompt+= f"and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. \n"

    system_prompt+= f"Respond in markdown. Include details of company culture, customers and careers/jobs if you have the information."

    return system_prompt

#print(get_brochure_system_prompt("funny"))


In [8]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [9]:
def create_brochure_system_prompt_options(company_name, url, mode):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": get_brochure_system_prompt(mode)},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

il faut que la fonction ne display pas le markdonwn du résultat mais le retourne pour le sauver

Ensuite, implémenter la traduction anthropic et l'image dall.e et l'introduire dans le modèle gradio généré par chatgpt (donc à vérifier)

In [22]:
result = create_brochure_system_prompt_options("Santé Publique France", "https://www.santepubliquefrance.fr/","funny")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.santepubliquefrance.fr/a-propos'}, {'type': 'news page', 'url': 'https://www.santepubliquefrance.fr/les-actualites'}, {'type': 'careers page', 'url': 'https://www.santepubliquefrance.fr/offres-d-emploi'}]}


```markdown
# 🎉 Welcome to Santé Publique France! 🎉

## 🤔 Who Are We?
At Santé Publique France, we’re like the health superheroes of France, but without the tight spandex suits (thankfully!). We handle everything from monitoring infectious diseases to promoting the importance of vaccines. Our motto: **“Prevent, Detect, Protect!”** We're passionate about making sure everyone stays healthy and informed. 

## 🕵️‍♀️ What Do We Do?
Here are some of the exhilarating missions we undertake:
- **Cancer Surveillance**: We’re the detectives of disease, tracking down the sneaky microbes behind cancers. Think of us as the Sherlock Holmes of cell biology!
- **Infection Alert Squad**: From STIs to foodborne illnesses and beyond, we’re on the front lines keeping unhealthy bacteria at bay. Don’t worry, we can’t hear you screaming from the other end of the microscope!
- **Vector Control**: Yes, we know all about those tiny buzzers (a.k.a mosquitoes) that bite more than just lip balm sales. We’re working hard to keep those pesky bugs from spoiling your summer fun!

## 📊 Data Wizards
With our magical tool **GÉODES**, we provide health data tighter than your best friend's jeans after the holidays! You can measure everything from cancer stats to antibiotic resistance. It's all about keeping our finger on the pulse—literally!

## 🌱 Company Culture
At Santé Publique France, our office is the ultimate healthy haven! We have yoga breaks to keep stress levels lower than a summer intern’s coffee budget. Our team shares everything: ideas, laughs, and occasional healthy snacks (carrots, but make them fancy). 

## 🧑‍💼 Careers at Santé Publique France
Looking for a job? Instead of just a paycheck, how about a chance to save lives while wearing comfy shoes? Here, if you’re passionate about public health and have a knack for adventure, we might just be your Cup of Health (better than a cup of tea)! Join us, and *“let’s battle diseases and have fun doing it!”*

## 🎯 Who's Our Target?
Our customers range from government entities to concerned citizens who want to keep germs at bay. Basically, if you breathe air, we’re here for you! We love working with all who seek to understand health better; whether you're a health enthusiast, a student or just someone who enjoys learning all about the marvels of the human body!

## 🔍 Why Choose Us?
So whether you’re looking to invest in your health, recruit some kick-butt talent, or just keep up-to-date with the latest in health trends, Santé Publique France is your best bet! With us, you’re not just part of a health agency—you’re part of a vibrant community focused on keeping France healthy and happy!

---

**Join the Santé Publique France family today—where health is our superpower!** 
```


In [38]:
create_brochure_system_prompt_options("Le Monde", "https://www.lemonde.fr/","serious")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.lemonde.fr/qui-sommes-nous/article/2007/11/17/talents-un-site-d-emploi-coedite-par-le-monde-interactif-et-telerama_978404_3386.html'}, {'type': 'careers page', 'url': 'https://www.lemonde.fr/le-monde-evenements/'}, {'type': 'company page', 'url': 'https://www.lemonde.fr/le-club-de-l-economie/'}]}


# Le Monde Brochure

---

## Company Overview

**Webpage Title:** Le Monde.fr - Actualités et Infos en France et dans le monde

Le Monde is a leading French newspaper renowned for its comprehensive coverage of national and international news. Founded with a commitment to providing high-quality journalism, Le Monde aims to keep its audience informed on critical issues in politics, society, economics, culture, and more.

---

## Company Culture

Le Monde prides itself on a culture of integrity, professionalism, and a strong sense of responsibility towards informing the public. The organization believes that a well-informed society is essential for democracy and strives to uphold journalistic standards that prioritize accuracy and balanced reporting. 

The editorial team works collaboratively, valuing diverse perspectives in its mission of delivering insightful analysis and exploration of global events. This emphasis not only fosters a robust working environment but also contributes to the credibility and reliability of the information presented.

---

## Customers

Le Monde serves a diverse audience, including:

- **General readers** seeking timely news coverage and analysis.
- **Subscribers** who receive exclusive content, including investigative reports, commentaries, and features.
- **Students and scholars** who utilize the paper for academic research and cultural insights.

The newspaper has continuously adapted its services to meet the needs of digital consumers and maintain its relevance in a rapidly evolving media landscape.

---

## Careers at Le Monde

Le Monde is not only a pillar of journalism but also a vibrant workplace for individuals passionate about media. The organization offers a variety of career opportunities in different fields, including:

- **Journalism**: Reporting, editing, and investigative roles that seek curious and dedicated individuals.
- **Digital and Tech**: Opportunities in digital strategy, product management, and technology integration for those looking to innovate in the news space.
- **Marketing and Business Development**: Roles focused on broadening reach and enhancing subscriber experience.

Le Monde values talent and dedication, offering a supportive work environment where new ideas are encouraged. The newsroom actively promotes continuous learning and development, providing employees with the resources needed to advance in their careers.

---

## Conclusion

With its unwavering commitment to quality journalism and a nurturing work culture, Le Monde represents a leader in the media landscape of France. Whether you are a reader, a potential subscriber, an investor interested in media ventures, or a candidate looking to build a career in journalism, Le Monde remains an exemplary choice for those seeking depth, accuracy, and integrity in news.

---

For more information, visit [Le Monde.fr](https://www.lemonde.fr).

## Translation

Asking Claude to translate the brochure in French, being careful to keep the general tone of the speech.

## Creating an image

Using cheap dall.e, must attract applications.

After the first version of the gradio app is created

## gradio

In [ ]:
import gradio as gr

# Fonction pour générer un texte basé sur les inputs
#def generate_text(organization_name, organization_url, option):
#    return f"Organisation: {organization_name}\nURL: {organization_url}\nOption choisie: {option}"

# Fonction pour générer une image (ici une image d'exemple)
def generate_image(text):
    # Ici vous pouvez créer l'image selon le texte, pour l'exemple nous utilisons une image fixe.
    # Par exemple, une image qui pourrait être générée avec la bibliothèque PIL, OpenCV, etc.
    # Pour l'exemple, une image vide est retournée.
    from PIL import Image, ImageDraw, ImageFont
    
    # Créer une image vide avec du texte
    img = Image.new('RGB', (200, 200), color = (255, 255, 255))
    d = ImageDraw.Draw(img)
    d.text((10, 10), text, fill=(0, 0, 0))
    
    return img

# Fonction de réinitialisation
def clear_fields():
    return "", "", "", None

# Création de l'interface Gradio
with gr.Blocks() as ui:
    with gr.Row():
        # Champs d'entrée pour le nom de l'organisation et l'URL
        entry_name = gr.Textbox(label="Nom de l'organisation")
        entry_url = gr.Textbox(label="URL de l'organisation")
        
    with gr.Row():
        # Dropdown pour choisir une option
        option_dropdown = gr.Dropdown(["Option 1", "Option 2"], label="Choisissez une option")
    
    with gr.Row():
        # Zone de texte pour afficher le texte généré à gauche
        output_text = gr.Textbox(label="Texte généré", interactive=False, elem_id="output_text")
    
    with gr.Row():
        # Zone d'image pour afficher l'image générée à droite
        output_image = gr.Image(label="Image générée", interactive=False, elem_id="output_image")
    
    with gr.Row():
        # Bouton pour générer texte et image
        submit_button = gr.Button("Générer")
        # Bouton pour réinitialiser les champs
        clear_button = gr.Button("Clear")
    
    # Connexion des boutons aux fonctions
    submit_button.click(
        fn=lambda org_name, org_url, option : (create_brochure_system_prompt_options(org_name, org_url, option), 
                                               generate_image(generate_text(org_name, org_url, option))),
        inputs=[entry_name, entry_url, option_dropdown],
        outputs=[output_text, output_image]
    )
    
    clear_button.click(fn=clear_fields, outputs=[entry_name, entry_url, option_dropdown, output_image])

# Lancer l'interface Gradio
ui.launch()
